# Extracting player information

- For a comparision, the 1st match has an id='335982' and it's scoreboard can be found in the [cricinfo page.](https://www.espncricinfo.com/series/8048/scorecard/335982/royal-challengers-bangalore-vs-kolkata-knight-riders-1st-match-indian-premier-league-2007-08)

- The ball-by-ball data can be downloaded from (https://cricsheet.org/downloads/).



In [1]:
import numpy as np
import pandas as pd
import pickle

from utils import get_match_list
from print_scorecard import print_scorecard

data_dir='../datasets/ipl/'

In [2]:
import psycopg2

# load the postgress credentials
import sys
sys.path.insert(0, '/Users/gshyam/creds/')
from load_credentials import load_credentials
fil='/Users/gshyam/creds/postgress.dat'
my_info = load_credentials(fil)

conn = psycopg2.connect(
    host="127.0.0.1",
    database="myfirstdb",
    user = my_info["user"],
    password = my_info["password"]
)
# cursor
c = conn.cursor()

## Generalize the submission

In [3]:
col_names = ['batsman', 'zeros', 'ones', 'twos', 'threes', 'fours',
             'fives', 'sixes', 'runs', 'balls_faced', 'not_out', 'team', 'against',
             'tied', 'win', 'toss', 'team_total', 'season', 'date', 'match_id']
col_types = ['text', 'int', 'int', 'int', 'int', 'int',
             'int', 'int', 'int', 'int', 'boolean', 'text', 'text',
             'boolean', 'boolean', 'boolean', 'int', 'int', 'text', 'int']

#for i in range(len(col_names)):
#    print ("'"+col_names[i]+"' : '"+col_types[i]+"',")

In [4]:
col_dict = {'batsman' : 'text',
            'zeros' : 'int',
            'ones' : 'int',
            'twos' : 'int',
            'threes' : 'int',
            'fours' : 'int',
            'fives' : 'int',
            'sixes' : 'int',
            'runs' : 'int',
            'balls_faced' : 'int',
            'not_out' : 'boolean',
            'team' : 'text',
            'against' : 'text',
            'tied' : 'boolean',
            'win' : 'boolean',
            'toss' : 'boolean',
            'team_total' : 'int',
            'season' : 'int',
            'date' : 'text',
            'match_id' : 'int'}
    
def create_table_full(table_name, col_dict=col_dict):
    #pd.read_sql(cmd, con) # This command doesn't work in creating empty table
    cmd = "CREATE TABLE  "+table_name+" ( "
    for k, v in col_dict.items():
        cmd = cmd+ k +" "+ v +", "
    cmd = cmd[:-2] + " )"  # :-2 to get rid of last comma
    c.execute(cmd)
    conn.commit()

#table_name = 'gri0'
#create_table_full(table_name)

In [7]:
def push_to_database_(match_id, table_name):
    df_bat, df_bowl = print_scorecard(str(match_id)+'.yaml', data_dir=data_dir)
    batters = df_bat.batsman.unique()
    for batter in batters:
        data = df_bat[df_bat['batsman']==batter].values[0]        

        cmd = "INSERT INTO "+ table_name+" VALUES ("
        for i, (k, v) in enumerate(col_dict.items()):
            if v == 'text':
                cmd = cmd + "'"+data[i]+"', "
            else:
                cmd = cmd + str(data[i]) +", "
                
        cmd = cmd[:-2]+")" # -2 to get rid of comma from the last entry
        #print (cmd)
        c.execute(cmd)
        conn.commit()

match_id = 335982
push_to_database_(match_id, table_name)

INSERT INTO gri0 VALUES ('SC Ganguly', 8, 2, 0, 0, 2, 0, 0, 10, 12, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO gri0 VALUES ('BB McCullum', 23, 22, 9, 0, 10, 0, 13, 158, 73, True, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO gri0 VALUES ('RT Ponting', 9, 8, 1, 0, 1, 0, 1, 20, 20, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO gri0 VALUES ('DJ Hussey', 4, 6, 1, 0, 1, 0, 0, 12, 12, False, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO gri0 VALUES ('Mohammad Hafeez', 1, 1, 0, 0, 1, 0, 0, 5, 3, True, 'Kolkata Knight Riders', 'Royal Challengers Bangalore', False, True, False, 222, 2008, '2008-04-18', 335982)
INSERT INTO gri0 VALUES ('R Dravid', 1, 2, 0, 0, 0, 0, 0, 2, 3, False, 'Ro

In [5]:
#def fetch_all_data(table_name):
#    c.execute("SELECT * FROM "+table_name)
#    return pd.DataFrame(c.fetchall())

def fetch_all_data(table_name):
    cmd = "SELECT * FROM "+table_name
    return pd.read_sql(cmd, conn)

#cmd = "SELECT * FROM "+table_name
#dff = pd.read_sql(cmd, conn)

table_name = 'gri0'
dff = fetch_all_data(table_name)
print ( dff.shape )
dff.head(2)

(16, 20)


,batsman,zeros,ones,twos,threes,fours,fives,sixes,runs,balls_faced,not_out,team,against,tied,win,toss,team_total,season,date,match_id
0,SC Ganguly,8,2,0,0,2,0,0,10,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
1,BB McCullum,23,22,9,0,10,0,13,158,73,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982


### Real Table
**which should look like actual table**

In [18]:
table_name = 'realtable'
create_table_full(table_name)

In [25]:
def push_all(table_name, years = [2008]):
    #years=range(2008, 2009, 1)
    for year in years:
        df_match=get_match_list(year, data_dir=data_dir)
        match_ids = df_match['match_id']
        
        for match_id in match_ids:
            df_bat, df_bowl = print_scorecard(str(match_id)+'.yaml', data_dir=data_dir)
            batters = df_bat.batsman.unique()
            for batter in batters:
                data = df_bat[df_bat['batsman']==batter].values[0]

                cmd = "INSERT INTO "+ table_name+" VALUES ("
                
                for i, (k, v) in enumerate(col_dict.items()):
                    if v == 'text':
                        cmd = cmd + "'"+data[i]+"', "
                    else:
                        cmd = cmd + str(data[i]) +", "
                cmd = cmd[:-2]+")" # -2 to get rid of comma from the last entry
                c.execute(cmd)
                conn.commit()
        print (year, "Done.")

#push_all(table_name)

In [20]:
table_name = 'realtable'
df = fetch_all_data(table_name)
df.head(2)

,batsman,zeros,ones,twos,threes,fours,fives,sixes,runs,balls_faced,not_out,team,against,tied,win,toss,team_total,season,date,match_id
0,SC Ganguly,8,2,0,0,2,0,0,10,12,False,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982
1,BB McCullum,23,22,9,0,10,0,13,158,73,True,Kolkata Knight Riders,Royal Challengers Bangalore,False,True,False,222,2008,2008-04-18,335982


In [21]:
table_name = 'realtable_810'
create_table_full(table_name)

In [26]:
years = [2008, 2009, 2010]
push_all(table_name, years = years)

2008 Done.
2009 Done.
2010 Done.


In [19]:
cmd = "SELECT * FROM "+table_name+ " WHERE sixes >0"
df_sixes = pd.read_sql(cmd, conn)
print ( df_sixes.sixes.values)

[13  1  1  2  1  2  1  1  9  3  1  2  1  3  2  2  1  2  2  1  1  3  1  1
  1  1  2  2  1  1  2  2  1  3  1  1  5  1  1  4  1  6  2  3  1  1  1  1
  1  1  3  1  7  1  1  2  6  3  2  1  1  1  1  1  1  1  1  1  2  1  1  3
  2  3  1  3  2  1  1  1  2  1  3  1 10  1  1  2  2  3  2  3  1  2  1  1
  1  1  3  2  1  1  1  2  1  1  2  1  2  2  2  2  1  1  1  1  1  2  1  1
  4  2  2  3  2  1  1  1  1  1  4  4  1  1  1  1  2  2  3  3  1  1  1  1
  1  1  2  1  2  2  1  1  1  2  1  1  1  1  1  3  1  1  1  1  1  2  1  3
  1  1  1  1  2  3  4  4  1  3  1  1  1  4  5  1  5  3  1  2  6  1  1  2
  3  2  1  2  2  1 11  1  3  2  1  2  2  2  2  1  1  3  2  1  6  5  1  3
  3  2  1  2  2  2  2  3  3  1  1  4  1  1  1  1  2  1  1  1  1  1  3  1
  2  4  3  1  2  1  2  1  1  3  4  1  3  2  1  1  1  4  2  3  2  1  2  4
  5  2  1  1  1  3  2  2  1  6  3  2  2  2  1  1  1  2  3  1  1  1  1  1
  1  2  1  7  1  6  1  3  3  1  1  1  4  1  1  1  3  1  4  1  1  1  1  4
  1  2  2  2  4  1]


In [16]:
cmd = "SELECT * FROM "+table_name+" WHERE sixes>0"
dff = c.execute(cmd)

In [29]:
def fetch_condition(table_name):
    cmd = "SELECT * FROM "+table_name+" WHERE sixes>0"
    print (cmd)
    c.execute(cmd)
    #rows = c.fetchall()
    #df = pd.DataFrame(rows)
    #display (df.head(5))
fetch_condition(table_name)

SELECT * FROM realtable WHERE batsman='MS Dhoni' 


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
